In [ ]:
import pandas as pd
import shutil, os, subprocess
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/NIH/')

Mounted at /content/drive


In [ ]:
dtype_gmdn = {'PrimaryDI': str}  # Making PrimaryDI as string type
dtype_sizes = {'PrimaryDI': str}  # Making PrimaryDI as string type

# Read the file into a DataFrame
df = pd.read_csv('gmdnTerms.txt', sep='|', dtype=dtype_gmdn)
sizes_df = pd.read_csv('deviceSizes.txt', sep='|', dtype=dtype_sizes)

# Display the first few rows of the DataFrame
# print(devsizes_df)

# # Remove rows where sizeType is 'Device Size Text, specify'
# sizes_df = sizes_df[sizes_df['sizeType'] != 'Device Size Text, specify']

<ipython-input-2-ae9981c4b0bf>:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sizes_df = pd.read_csv('deviceSizes.txt', sep='|', dtype=dtype_sizes)


In [ ]:
# Get the column names
headers = df.columns.tolist()

# Print the column names
print(headers)

['PrimaryDI', 'gmdnPTName', 'gmdnPTDefinition', 'gmdnCode', 'gmdnCodeStatus', 'implantable']


In [ ]:
# Get sizes df column names
size_headers = sizes_df.columns.tolist()

# Print the sizes df column names
print(size_headers)

['PrimaryDI', 'sizeType', 'size (Unit)', 'size (Value)', 'sizeText']


In [ ]:
sizes_df["size (Unit)"].unique()

array(['Millimeter', 'Centimeter', 'Micrometer', nan, 'Milliliter',
       'Gauge', 'French', 'Square meter', 'Inch', 'Liter', 'Kilogram',
       'Square centimeter', 'degree', 'Gram', 'Meter', 'Feet', 'Pound',
       'Milligram', 'Nanometer', 'Fluid Ounce', 'Square millimeter',
       'Gallon', 'Yard', 'Centiliter', 'Square inch', 'Quart', 'Pint',
       'Microliter', 'Cubic Inch', 'millibar', 'Pound per Square Inch',
       'Atmosphere', 'Femtometer', 'Decimeter', 'Kilometer', 'Microgram',
       'Deciliter', 'Hertz', 'KiloPascal'], dtype=object)

In [ ]:
sizes_df["sizeType"].unique()

array(['Width', 'Length', 'Outer Diameter', 'Lumen/Inner Diameter',
       'Height', 'Pore Size', 'Device Size Text, specify', 'Total Volume',
       'Needle Gauge', 'Catheter Gauge', 'Area/Surface Area', 'Depth',
       'Weight', 'Angle', 'Guidewire Length', 'Shaft Length',
       'Catheter Length', 'Guidewire Diameter', 'Circumference',
       'Catheter Working Length', 'Tip Bend Radius', 'Pressure',
       'Balloon Diameter', 'Balloon Length', 'Stent Length',
       'Catheter Inner Diameter', 'Introducer Sheath Compatibility',
       'Crossing Profile', 'Balloon Rated Burst Pressure',
       'Stent Diameter', 'Balloon Catheter Tip Length',
       'Balloon Proximal Outer Diameter (OD)',
       'Balloon Nominal (Inflation) Pressure', 'Guidewire Compatibility',
       'Atherectomy Rotating Component Diameter',
       'Maximum Stent Diameter', 'Atherectomy Cutter Diameter'],
      dtype=object)

In [ ]:
from tqdm import tqdm

# Get unique sizeTypes
unique_size_types = sizes_df['sizeType'].unique()

# Initialize the new DataFrame
new_df = pd.DataFrame(columns=['PrimaryDI'] + list(unique_size_types))
new_df['PrimaryDI'] = sizes_df['PrimaryDI'].unique()
new_df = new_df.set_index('PrimaryDI')

# Populate the new DataFrame
for idx, row in tqdm(sizes_df.iterrows(), total=len(sizes_df), desc="Processing Rows"):
    primary_di = row['PrimaryDI']
    size_type = row['sizeType']

    if size_type == "Device Size Text, specify":
      size_text = row['sizeText']
      new_df.loc[primary_di, size_type] = f"{size_text}"

    size_value = row['size (Value)']
    size_unit = row['size (Unit)']

    new_df.loc[primary_di, size_type] = f"{size_value} {size_unit}"

# Reset the index to make 'PrimaryDI' a normal column
new_df.reset_index(inplace=True)

Processing Rows: 100%|██████████| 1614597/1614597 [04:48<00:00, 5605.72it/s]


In [ ]:
new_df = new_df.rename(columns={'Device Size Text, specify': 'Size Text'})

In [ ]:
new_df.fillna('N/A', inplace=True)

print(new_df.iloc[0])

PrimaryDI                                  04048551113073
Width                                      10  Millimeter
Length                                     28  Millimeter
Outer Diameter                                        N/A
Lumen/Inner Diameter                                  N/A
Height                                                N/A
Pore Size                                             N/A
Size Text                                             N/A
Total Volume                                          N/A
Needle Gauge                                          N/A
Catheter Gauge                                        N/A
Area/Surface Area                                     N/A
Depth                                                 N/A
Weight                                                N/A
Angle                                                 N/A
Guidewire Length                                      N/A
Shaft Length                                          N/A
Catheter Lengt

In [ ]:
specific_row = new_df[new_df['PrimaryDI'] == '00817956022808'].iloc[0]

print(specific_row)

PrimaryDI                                  00817956022808
Width                                        11 Kilometer
Length                                                N/A
Outer Diameter                                        N/A
Lumen/Inner Diameter                                  N/A
Height                                            12 Inch
Pore Size                                             N/A
Size Text                                             N/A
Total Volume                                          N/A
Needle Gauge                                          N/A
Catheter Gauge                                        N/A
Area/Surface Area                                     N/A
Depth                                              4 Inch
Weight                                                N/A
Angle                                                 N/A
Guidewire Length                                      N/A
Shaft Length                                          N/A
Catheter Lengt

In [ ]:
new_df

,PrimaryDI,Width,Length,Outer Diameter,Lumen/Inner Diameter,Height,Pore Size,Size Text,Total Volume,Needle Gauge,...,Crossing Profile,Balloon Rated Burst Pressure,Stent Diameter,Balloon Catheter Tip Length,Balloon Proximal Outer Diameter (OD),Balloon Nominal (Inflation) Pressure,Guidewire Compatibility,Atherectomy Rotating Component Diameter,Maximum Stent Diameter,Atherectomy Cutter Diameter
0,04048551113073,10 Millimeter,28 Millimeter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,04048551113035,10 Millimeter,23 Millimeter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,04048551112984,9 Millimeter,28 Millimeter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,04048551112939,9 Millimeter,25 Millimeter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,04048551112922,9 Millimeter,23 Millimeter,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870704,EFORC7060849,N/A,N/A,N/A,N/A,0.018 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
870705,EFORC7060848,N/A,N/A,N/A,N/A,0.018 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
870706,EFORC7060847,N/A,N/A,N/A,N/A,0.018 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
870707,EFORC7060843,N/A,N/A,N/A,N/A,0.018 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [ ]:
name_counts = df['gmdnCode'].value_counts()

# Set display options to show more rows
pd.set_option('display.max_rows', len(name_counts))

name_counts

,count
gmdnCode,
61325,280849
33961,243292
30878,92692
38643,81495
60762,70677
16187,68318
35065,59930
44879,58468
44057,57861


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Define the model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Check if a GPU is available and move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize and generate embeddings
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().detach().numpy()

# Function to create a descriptive prompt from a row
def create_prompt(row):
    prompt = "; ".join([f"{col} = {row[col]}" for col in row.index])
    return prompt

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BeRT Results

In [ ]:
# for gmdncode in tqdm(dict(name_counts).keys(), desc="Processing gmdnCodes"):
gmdncode = 37480

# Filter df to get PrimaryDI values where gmdnCode equals the current gmdncode
filtered_primary_di = df[df['gmdnCode'] == gmdncode]['PrimaryDI']

# Convert to list
filtered_primary_di_list = filtered_primary_di.tolist()

# Filter new_df based on PrimaryDI values obtained from df
filtered_sizes_df = new_df[new_df['PrimaryDI'].isin(filtered_primary_di_list)].copy()  # Use .copy() to avoid SettingWithCopyWarning

# if filtered_sizes_df.shape[0] < 20:
#     continue
print(f"Dataset Size = {filtered_sizes_df.shape[0]}\n")

# print(filtered_sizes_df)

# Extract 'PrimaryDI' and the feature columns
primary_di = filtered_sizes_df['PrimaryDI']
features = filtered_sizes_df.drop(columns=['PrimaryDI'])

# Convert structured data into string format
features_str = features.applymap(str)

# Generate embeddings for each row
embeddings = []
for _, row in features_str.iterrows():
    prompt = create_prompt(row)
    embeddings.append(generate_embeddings(prompt))

embeddings = np.vstack(embeddings)

print(f"Num of embeddings = {embeddings.shape}\n")

# Scale the data
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)
# embeddings_scaled = embeddings_scaled

# Fit the Isolation Forest model
iso_forest = IsolationForest(contamination=0.1, random_state=42)  # Adjust contamination based on expected anomaly percentage
iso_forest.fit(embeddings_scaled)

# Predict anomalies
anomaly_labels = iso_forest.predict(embeddings_scaled)
anomaly_scores = iso_forest.decision_function(embeddings_scaled)

# Add the results to the DataFrame
filtered_sizes_df.loc[:, 'anomaly_score'] = anomaly_scores
filtered_sizes_df.loc[:, 'anomaly_label'] = anomaly_labels

# Add gmdnCode to the DataFrame
filtered_sizes_df.loc[:, 'gmdnCode'] = gmdncode

# Filter out potential anomalies (anomaly_label = -1 indicates anomaly)
anomalies = filtered_sizes_df[filtered_sizes_df['anomaly_label'] == -1]

# print(anomalies)
# # Write the anomalies to the CSV file
# if not anomalies.empty:
#     anomalies.to_csv('detected_anomalies_llm_embeddings.csv', mode='a', header=not pd.io.common.file_exists('detected_anomalies_llm_embeddings.csv'), index=False)

# # Print the anomalies
# all_anomalies_df = pd.read_csv('detected_anomalies_llm_embeddings.csv')
# all_anomalies_df
filtered_sizes_df

Dataset Size = 75



<ipython-input-22-e59f5c5d4c5a>:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features_str = features.applymap(str)


Num of embeddings = (75, 768)



,PrimaryDI,Width,Length,Outer Diameter,Lumen/Inner Diameter,Height,Pore Size,Size Text,Total Volume,Needle Gauge,...,Balloon Catheter Tip Length,Balloon Proximal Outer Diameter (OD),Balloon Nominal (Inflation) Pressure,Guidewire Compatibility,Atherectomy Rotating Component Diameter,Maximum Stent Diameter,Atherectomy Cutter Diameter,anomaly_score,anomaly_label,gmdnCode
348185,00817956023201,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.038426,1,37480
348186,00817956023195,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.038426,1,37480
348187,00817956023188,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.038426,1,37480
348188,00817956023171,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.038426,1,37480
348189,00817956023164,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.068914,1,37480
348190,00817956023157,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.068914,1,37480
348191,00817956023140,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.068914,1,37480
348192,00817956023133,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.068914,1,37480
348222,00817956022839,15 Inch,N/A,N/A,N/A,18 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.065543,1,37480
348223,00817956022822,15 Inch,N/A,N/A,N/A,18 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.065543,1,37480


In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.9 MB/s eta 0:00:00


SBeRT light results

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Define the SBERT model
model_name = "all-MiniLM-L6-v2"  # You can change this to any other SBERT model
model = SentenceTransformer(model_name)

# Check if a GPU is available and move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Generate embeddings using SBERT
def generate_embeddings(text):
    embeddings = model.encode(text, convert_to_tensor=True, device=device)
    return embeddings.cpu().detach().numpy()

# Function to create a descriptive prompt from a row
def create_prompt(row):
    prompt = "; ".join([f"{col} = {row[col]}" for col in row.index])
    return prompt

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# for gmdncode in tqdm(dict(name_counts).keys(), desc="Processing gmdnCodes"):
gmdncode = 37480

# Filter df to get PrimaryDI values where gmdnCode equals the current gmdncode
filtered_primary_di = df[df['gmdnCode'] == gmdncode]['PrimaryDI']

# Convert to list
filtered_primary_di_list = filtered_primary_di.tolist()

# Filter new_df based on PrimaryDI values obtained from df
filtered_sizes_df = new_df[new_df['PrimaryDI'].isin(filtered_primary_di_list)].copy()  # Use .copy() to avoid SettingWithCopyWarning

# if filtered_sizes_df.shape[0] < 20:
#     continue
print(f"Dataset Size = {filtered_sizes_df.shape[0]}\n")

# print(filtered_sizes_df)

# Extract 'PrimaryDI' and the feature columns
primary_di = filtered_sizes_df['PrimaryDI']
features = filtered_sizes_df.drop(columns=['PrimaryDI'])

# Convert structured data into string format
features_str = features.applymap(str)

# Generate embeddings for each row
embeddings = []
for _, row in features_str.iterrows():
    prompt = create_prompt(row)
    embeddings.append(generate_embeddings(prompt))

embeddings = np.vstack(embeddings)

print(f"Num of embeddings = {embeddings.shape}\n")

# Scale the data
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)
# embeddings_scaled = embeddings_scaled

# Fit the Isolation Forest model
iso_forest = IsolationForest(contamination=0.1, random_state=42)  # Adjust contamination based on expected anomaly percentage
iso_forest.fit(embeddings_scaled)

# Predict anomalies
anomaly_labels = iso_forest.predict(embeddings_scaled)
anomaly_scores = iso_forest.decision_function(embeddings_scaled)

# Add the results to the DataFrame
filtered_sizes_df.loc[:, 'anomaly_score'] = anomaly_scores
filtered_sizes_df.loc[:, 'anomaly_label'] = anomaly_labels

# Add gmdnCode to the DataFrame
filtered_sizes_df.loc[:, 'gmdnCode'] = gmdncode

# Filter out potential anomalies (anomaly_label = -1 indicates anomaly)
anomalies = filtered_sizes_df[filtered_sizes_df['anomaly_label'] == -1]

# print(anomalies)
# # Write the anomalies to the CSV file
# if not anomalies.empty:
#     anomalies.to_csv('detected_anomalies_llm_embeddings.csv', mode='a', header=not pd.io.common.file_exists('detected_anomalies_llm_embeddings.csv'), index=False)

# # Print the anomalies
# all_anomalies_df = pd.read_csv('detected_anomalies_llm_embeddings.csv')
# all_anomalies_df
filtered_sizes_df

Dataset Size = 75



<ipython-input-25-e59f5c5d4c5a>:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features_str = features.applymap(str)


Num of embeddings = (75, 384)



,PrimaryDI,Width,Length,Outer Diameter,Lumen/Inner Diameter,Height,Pore Size,Size Text,Total Volume,Needle Gauge,...,Balloon Catheter Tip Length,Balloon Proximal Outer Diameter (OD),Balloon Nominal (Inflation) Pressure,Guidewire Compatibility,Atherectomy Rotating Component Diameter,Maximum Stent Diameter,Atherectomy Cutter Diameter,anomaly_score,anomaly_label,gmdnCode
348185,00817956023201,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.108844,1,37480
348186,00817956023195,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.108844,1,37480
348187,00817956023188,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.108844,1,37480
348188,00817956023171,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.108844,1,37480
348189,00817956023164,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.085788,1,37480
348190,00817956023157,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.085788,1,37480
348191,00817956023140,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.085788,1,37480
348192,00817956023133,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.085788,1,37480
348222,00817956022839,15 Inch,N/A,N/A,N/A,18 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.047499,1,37480
348223,00817956022822,15 Inch,N/A,N/A,N/A,18 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.047499,1,37480


SBeRT heavy results

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Define the SBERT model
model_name = "all-mpnet-base-v2"  # You can change this to any other SBERT model
model = SentenceTransformer(model_name)

# Check if a GPU is available and move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Generate embeddings using SBERT
def generate_embeddings(text):
    embeddings = model.encode(text, convert_to_tensor=True, device=device)
    return embeddings.cpu().detach().numpy()

# Function to create a descriptive prompt from a row
def create_prompt(row):
    prompt = "; ".join([f"{col} = {row[col]}" for col in row.index])
    return prompt

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# for gmdncode in tqdm(dict(name_counts).keys(), desc="Processing gmdnCodes"):
gmdncode = 37480

# Filter df to get PrimaryDI values where gmdnCode equals the current gmdncode
filtered_primary_di = df[df['gmdnCode'] == gmdncode]['PrimaryDI']

# Convert to list
filtered_primary_di_list = filtered_primary_di.tolist()

# Filter new_df based on PrimaryDI values obtained from df
filtered_sizes_df = new_df[new_df['PrimaryDI'].isin(filtered_primary_di_list)].copy()  # Use .copy() to avoid SettingWithCopyWarning

# if filtered_sizes_df.shape[0] < 20:
#     continue
print(f"Dataset Size = {filtered_sizes_df.shape[0]}\n")

# print(filtered_sizes_df)

# Extract 'PrimaryDI' and the feature columns
primary_di = filtered_sizes_df['PrimaryDI']
features = filtered_sizes_df.drop(columns=['PrimaryDI'])

# Convert structured data into string format
features_str = features.applymap(str)

# Generate embeddings for each row
embeddings = []
for _, row in features_str.iterrows():
    prompt = create_prompt(row)
    embeddings.append(generate_embeddings(prompt))

embeddings = np.vstack(embeddings)

print(f"Num of embeddings = {embeddings.shape}\n")

# Scale the data
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)
# embeddings_scaled = embeddings_scaled

# Fit the Isolation Forest model
iso_forest = IsolationForest(contamination=0.1, random_state=42)  # Adjust contamination based on expected anomaly percentage
iso_forest.fit(embeddings_scaled)

# Predict anomalies
anomaly_labels = iso_forest.predict(embeddings_scaled)
anomaly_scores = iso_forest.decision_function(embeddings_scaled)

# Add the results to the DataFrame
filtered_sizes_df.loc[:, 'anomaly_score'] = anomaly_scores
filtered_sizes_df.loc[:, 'anomaly_label'] = anomaly_labels

# Add gmdnCode to the DataFrame
filtered_sizes_df.loc[:, 'gmdnCode'] = gmdncode

# Filter out potential anomalies (anomaly_label = -1 indicates anomaly)
anomalies = filtered_sizes_df[filtered_sizes_df['anomaly_label'] == -1]

# print(anomalies)
# # Write the anomalies to the CSV file
# if not anomalies.empty:
#     anomalies.to_csv('detected_anomalies_llm_embeddings.csv', mode='a', header=not pd.io.common.file_exists('detected_anomalies_llm_embeddings.csv'), index=False)

# # Print the anomalies
# all_anomalies_df = pd.read_csv('detected_anomalies_llm_embeddings.csv')
# all_anomalies_df
filtered_sizes_df

Dataset Size = 75



<ipython-input-27-e59f5c5d4c5a>:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features_str = features.applymap(str)


Num of embeddings = (75, 768)



,PrimaryDI,Width,Length,Outer Diameter,Lumen/Inner Diameter,Height,Pore Size,Size Text,Total Volume,Needle Gauge,...,Balloon Catheter Tip Length,Balloon Proximal Outer Diameter (OD),Balloon Nominal (Inflation) Pressure,Guidewire Compatibility,Atherectomy Rotating Component Diameter,Maximum Stent Diameter,Atherectomy Cutter Diameter,anomaly_score,anomaly_label,gmdnCode
348185,00817956023201,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.063265,1,37480
348186,00817956023195,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.063265,1,37480
348187,00817956023188,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.063265,1,37480
348188,00817956023171,19 Inch,N/A,N/A,N/A,24 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.063265,1,37480
348189,00817956023164,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.079923,1,37480
348190,00817956023157,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.079923,1,37480
348191,00817956023140,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.079923,1,37480
348192,00817956023133,13 Inch,N/A,N/A,N/A,13 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.079923,1,37480
348222,00817956022839,15 Inch,N/A,N/A,N/A,18 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.062092,1,37480
348223,00817956022822,15 Inch,N/A,N/A,N/A,18 Inch,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,0.062092,1,37480
